In [4]:
%reset -f
testID = 0

import os
import shutil

import pandas as pd
import numpy as np
import math as mt
import matplotlib.pyplot as plt
import seaborn as sns
from timeit import default_timer as timer

# Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils import plot_model

# SkLearn
import sklearn.preprocessing as SklPreProcessing
import sklearn.metrics as SklMetrics
from sklearn.preprocessing import normalize
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score


%matplotlib inline

In [5]:
class Scoring_Functions():
    #
    def __init__(self, y_pred, y_true):
        self.y_pred = y_pred
        self.y_true = y_true
    #
    def accuracy(self):
        # http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score
        return str(accuracy_score(self.y_true, self.y_pred) * 100) + "%"
    #
    def precision(self):
        # http: // scikit - learn.org / stable / modules / generated / sklearn.metrics.precision_score.html  # sklearn.metrics.precision_score
        return str(precision_score(self.y_true, self.y_pred, average='weighted')* 100) + '%'
    #
    def recall(self):
        # http://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html#sklearn.metrics.recall_score
        return str(recall_score(self.y_true, self.y_pred, average='weighted') * 100) + '%'
    #
    def f_measure(self):
        # http://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score
        return str(f1_score(self.y_true, self.y_pred, average='weighted') * 100) + '%'
    #
    def scoring_results(self):
        return "Accuracy: " + str(self.accuracy()) + "\nPrecision: " + str(self.precision()) + "\nRecall: " + str(self.recall()) + "\nFMeasure: " + str(self.f_measure())

In [83]:
class LSTMPredictor:
    ############################
    # MetaData
    __testID = None
    __cmdCount = 0
    __cmdHistory = ''
    __resultsRootDir = 'results'
    ############################
    # Dataset info
    __eegData = None
    __eegHeader = None
    __testDataIndex = 0
    __trainDataFileName = None
    __testDataFileName = None
    ############################
    # ML model info
    __model = None
    __trainTestMap = None
    ############################
   
    def __init__(self, testID, trainFilename, testFilename):
        self.__testID = str(testID).zfill(5)
        self.__createTestResultsDir()
        self.__trainDataFileName = trainFilename
        self.__testDataFileName = testFilename
        self.ResetState()
        
    def ResetState(self):
        tmpFileName = ''
        self.__cmdHistory = ''
        self.__cmdCount = 0
        self.__loadData()
    
    ###########################################################################
    # Plot methods
    def PlotFeatures(self, filePrefix=None, figWidth=20, figHeight=20):
        self.__addToHistory('PlotFeatures_filePrefix:' + str(filePrefix))
        fig = plt.figure()
        fig.set_figwidth(figWidth)
        fig.set_figheight(figHeight)
        
        featureCount = len(self.__eegHeader) - 1
        idxSubplot = 0
        
        for itr in range(featureCount):
            idxSubplot = idxSubplot + 1
            currAx = fig.add_subplot(featureCount + 1, 1, idxSubplot)
            currAx.grid()
            currAx.plot(self.__eegData[:, itr])
            currAx.set_title(self.__eegHeader[itr])
        
        idxSubplot = idxSubplot + 1
        currAx = fig.add_subplot(featureCount + 1, 1, idxSubplot)
        currAx.grid()
        currAx.plot(self.__eegData[:, -1])
        currAx.set_title(self.__eegHeader[-1])

        if filePrefix is None:
            plt.show()
        else:
            pathToFile = self.__resultsRootDir + "/" + self.__testID + "/" + filePrefix
            plt.savefig(pathToFile)
            
    def PlotBoxPlots(self, filePrefix=None, figWidth=20, figHeight=20):
        self.__addToHistory('PlotBoxPlots_filePrefix:' + str(filePrefix))
        fig = plt.figure()
        fig.set_figwidth(figWidth)
        fig.set_figheight(figHeight)
        
        ax1 = fig.add_subplot(2, 1, 1)
        ax1.set_title('Feature range: Full')
        ax1.boxplot(self.__eegData[:, :-1]
                    , sym='b.'
                    , vert=False
                    , whis='range'
                    , labels=self.__eegHeader[:-1]
                    , meanline=True
                    , showbox=True
                    , showfliers=True)
        
        ax2 = fig.add_subplot(2, 1, 2)
        ax2.set_title('Feature range: [5%, 95%]')
        ax2.boxplot(self.__eegData[:, :-1]
                    , sym='b.'
                    , vert=False
                    , whis=[5, 95]
                    , labels=self.__eegHeader[:-1]
                    , meanline=True
                    , showbox=True
                    , showfliers=False)
        
        if filePrefix is not None:
            pathToFile = self.__resultsRootDir + "/" + self.__testID + "/" + filePrefix
            plt.savefig(pathToFile)
            
    def PlotMeanDistribution(self, filePrefix=None, figWidth=20, figHeight=20):
        self.__addToHistory('PlotMeanDistribution_filePrefix:' + str(filePrefix))
        idxEyeClosed = np.where(self.__eegData[:, -1] == 1)[0]
        idxEyeOpened = np.where(self.__eegData[:, -1] == 0)[0]
        
        featureCount = len(self.__eegHeader) - 1
        xTick = np.array(range(featureCount))
        
        fig = plt.figure()
        fig.set_figwidth(figWidth)
        fig.set_figheight(figHeight)

        ax = fig.add_subplot(1, 1, 1)
        ax.grid()
        ax.set_title('Mean Distribution')
        ax.set_xticklabels(self.__eegHeader[0:-1])
        ax.set_xticks(xTick)

        ax.plot(xTick, self.__eegData[idxEyeOpened][:, 0:featureCount].mean(axis=0), 'bo')
        ax.plot(xTick, self.__eegData[idxEyeClosed][:, 0:featureCount].mean(axis=0), 'ro')
        ax.plot(xTick, self.__eegData[:, 0:featureCount].mean(axis=0), 'go')

        ax.legend(['Eye Open', 'Eye Closed', 'Both'])
        
        if filePrefix is not None:
            pathToFile = self.__resultsRootDir + "/" + self.__testID + "/" + filePrefix
            plt.savefig(pathToFile)

    def PlotStdDevDistribution(self, filePrefix=None, figWidth=20, figHeight=20):
        self.__addToHistory('PlotStdDevDistribution_filePrefix:' + str(filePrefix))
        idxEyeClosed = np.where(self.__eegData[:, -1] == 1)[0]
        idxEyeOpened = np.where(self.__eegData[:, -1] == 0)[0]
        
        featureCount = len(self.__eegHeader) - 1
        xTick = np.array(range(featureCount))
        
        fig = plt.figure()
        fig.set_figwidth(figWidth)
        fig.set_figheight(figHeight)

        ax = fig.add_subplot(1, 1, 1)
        ax.grid()
        ax.set_title('Standard Deviation Distribution')
        ax.set_xticklabels(self.__eegHeader[0:-1])
        ax.set_xticks(xTick)

        ax.plot(xTick, self.__eegData[idxEyeOpened][:, 0:featureCount].std(axis=0), 'bo')
        ax.plot(xTick, self.__eegData[idxEyeClosed][:, 0:featureCount].std(axis=0), 'ro')
        ax.plot(xTick, self.__eegData[:, 0:featureCount].std(axis=0), 'go')

        ax.legend(['Eye Open', 'Eye Closed', 'Both'])
        
        if filePrefix is not None:
            pathToFile = self.__resultsRootDir + "/" + self.__testID + "/" + filePrefix
            plt.savefig(pathToFile)
            
    def FeatureCorrelationMatrix(self, filePrefix=None, figWidth=20, figHeight=20):
        self.__addToHistory('FeatureCorrelationMatrix_filePrefix:' + str(filePrefix))
        # Compute the correlation matrix
        corr = pd.DataFrame(data=self.__eegData[:, :-1], columns=list(self.__eegHeader[:-1])).corr()
        
        # Generate a mask for the upper triangle
        mask = np.zeros_like(corr, dtype=np.bool)
        mask[np.triu_indices_from(mask)] = True
        
        # Set up the matplotlib figure
        fig = plt.figure()
        fig.set_figwidth(figWidth)
        fig.set_figheight(figHeight)
        
        ax = fig.add_subplot(1, 1, 1)
        ax.set_title("Feature correlation matrix")
        
        # Generate a custom diverging colormap
        cmap = sns.diverging_palette(240, 10, as_cmap=True)
        
        # Draw the heatmap
        sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1.0, vmin=-1.0, center=0,
                    square=True, linewidths=.5, cbar_kws={"shrink": .5})
        
        if filePrefix is not None:
            pathToFile = self.__resultsRootDir + "/" + self.__testID + "/" + filePrefix
            plt.savefig(pathToFile)

    ###########################################################################
    # Getters
    def GetFeatureNames(self):
        return self.__eegHeader[:-1]
    
    def GetCmdHistory(self):
        return self.__cmdHistory
    ###########################################################################
    # Dataset operations
    def RemoveFeatures(self, indices):
        assert isinstance(indices, list), 'Parameter must be a list'

        self.__addToHistory('RemoveFeatures:' + ','.join(str(x) for x in indices))
        self.__eegHeader = np.delete(self.__eegHeader, indices)
        self.__eegData = np.delete(self.__eegData, indices, axis=1)
    
    def RemoveOutliers(self, upperLimit=None, lowerLimit=None):
        outLierIndexes = set()
        result = {}
        result['upper'] = set()
        result['lower'] = set()
        
        self.__addToHistory('RemoveOutliers_Upper:' + str(upperLimit)
                            + '_Lower:' + str(lowerLimit))
        featureCount = len(self.__eegHeader) - 1

        if upperLimit is not None:
            for itr in range(featureCount):
                outLiers = np.where(self.__eegData[:, itr] > upperLimit)[0]
                if len(outLiers) > 0:
                    for i in xrange(len(outLiers)):
                        result['upper'].add(outLiers[i])

        if lowerLimit is not None:
            for itr in range(featureCount):
                outLiers = np.where(self.__eegData[:, itr] < lowerLimit)[0]
                if len(outLiers) > 0:
                    for i in xrange(len(outLiers)):
                        result['lower'].add(outLiers[i])
                        
        deleteIndexes = list(result['upper'].union(result['lower']))
        self.__eegData = np.delete(self.__eegData, deleteIndexes, axis=0)
        
        return result

    def NormalizeData(self, normFunc='MinMax'):
        self.__addToHistory('NormalizeData_Func:' + normFunc)
        
        if normFunc == 'MinMax':
            scaler = SklPreProcessing.MinMaxScaler(feature_range=(0, 1))
            self.__eegData = scaler.fit_transform(self.__eegData)
        else:
            self.__eegData = normalize(self.__eegData, norm=normFunc)
        
    ###########################################################################
    # Keras LSTM model
    def CompileModel(self, lagCount, neuronCount, dropOut,
                     modelArchitectureFilename,
                     lstmStackCount=1, useLaggedOutput=True,
                     lossFunc='mae', optimizerFunc='adam'):
        assert lstmStackCount > 0, 'lstmStackCount must be > 0'
        self.__addToHistory('CompileModel_Lag:' + str(lagCount)
                            + '_NeuronCount:' + str(neuronCount)
                            + '_DropOut:' + str(dropOut)
                            + '_useLaggedOutput:' + str(useLaggedOutput)
                            + '_LossFunc:' + str(lossFunc)
                            + '_OptimizerFunc:' + str(optimizerFunc)
                            + '_lstmStackCount:' + str(lstmStackCount))
        
#         supervisedDataset = self.__timeSeriesToSupervised(lagCount, useLaggedOutput)
        featureCount = len(self.__eegHeader) - 1
        
#         values = supervisedDataset.values
#         trainingSet = values[:self.__testDataIndex, :]
#         testingSet = values[self.__testDataIndex:, :]

        trainingSet = self.__eegData[:self.__testDataIndex, :]
        testingSet = self.__eegData[self.__testDataIndex:, :]
        
        print testingSet.shape
        
#         pastInputCount = (featureCount + 1) * lagCount
        pastInputCount = featureCount
        self.__trainTestMap = {}

        self.__trainTestMap['trainingSet_inFeatures'] = trainingSet[:, :pastInputCount]
        self.__trainTestMap['trainingSet_outFeature'] = trainingSet[:, -1]
        
        self.__trainTestMap['testingSet_inFeatures'] = testingSet[:, :pastInputCount]
        self.__trainTestMap['testingSet_outFeature'] = testingSet[:, -1]
        
        # Reshape input to be 3D [samples, timesteps, features]
#         featureDimSize = (featureCount + 1)
        featureDimSize = featureCount
        
        print self.__trainTestMap['trainingSet_inFeatures'].shape
        
        self.__trainTestMap['trainingSet_inFeatures_reShaped'] = self.__trainTestMap['trainingSet_inFeatures'].reshape(
            (self.__trainTestMap['trainingSet_inFeatures'].shape[0], lagCount, featureDimSize))
        
        self.__trainTestMap['testingSet_inFeatures_reShaped'] = self.__trainTestMap['testingSet_inFeatures'].reshape(
            (self.__trainTestMap['testingSet_inFeatures'].shape[0], lagCount, featureDimSize))
        
        # Design network
        self.__model = Sequential()
        
        batch_size = 10000
        self.__model.add(LSTM(4, batch_input_shape=(batch_size, 1, 14), stateful=True, return_sequences=True))
        self.__model.add(LSTM(4, batch_input_shape=(batch_size, 1, 14), stateful=True))

        self.__model.add(Dense(1))
        self.__model.compile(loss=lossFunc, optimizer=optimizerFunc)
        
        # Visualize LSTM network
        pathToFile = self.__resultsRootDir + "/" + self.__testID + "/" + modelArchitectureFilename
        plot_model(self.__model, to_file=pathToFile, show_shapes=True)
        
    def FitModel(self, epochCount, batchSize,
                 resultsFile,
                 lossFuncFilePrefix=None,
                 predictionFilePrefix=None,
                 figWidth=20, figHeight=20, verbosity=2):
        assert self.__model != None, 'Model not compiled'
        
        self.__addToHistory('FitModel_EpochCount:' + str(epochCount)
                            + '_BatchSize:' + str(batchSize)
                            + '_lossFuncFilePrefix:' + str(lossFuncFilePrefix)
                            + '_predictionFilePrefix:' + str(predictionFilePrefix))
        
        # Fit model
        start = timer()
        loss = []

        for i in range(100):
            modelHistory = self.__model.fit(self.__trainTestMap['trainingSet_inFeatures_reShaped'],
                                     self.__trainTestMap['trainingSet_outFeature'],
                                     epochs=1, batch_size=1, verbose=2, shuffle=False)
            self.__model.reset_states()

            loss.extend(modelHistory.history['loss'])
            
#         history = self.__model.fit(self.__trainTestMap['trainingSet_inFeatures_reShaped'],
#                                  self.__trainTestMap['trainingSet_outFeature'],
#                                  epochs=epochCount,
#                                  batch_size=batchSize,
# #                                  validation_data=(
# #                                      self.__trainTestMap['testingSet_inFeatures_reShaped'],
# #                                      self.__trainTestMap['testingSet_outFeature']
# #                                  ),
#                                  verbose=verbosity, shuffle=False)
        end = timer()
        
        fig = plt.figure()
        fig.set_figwidth(figWidth)
        fig.set_figheight(figHeight)

        ax = fig.add_subplot(1, 1, 1)
        ax.grid()
        ax.set_title('Loss Function')
        ax.plot(loss)
#         ax.plot(history.history['loss'], label='train')
#         ax.plot(history.history['val_loss'], label='test')
        plt.legend(['train', 'test'])
                    
        if lossFuncFilePrefix is not None:
            pathToFile = self.__resultsRootDir + "/" + self.__testID + "/" + lossFuncFilePrefix
            plt.savefig(pathToFile)

        testDataPrediction = self.__model.predict_classes(self.__trainTestMap['testingSet_inFeatures_reShaped'])
        testDataPredictionRaw = self.__model.predict(self.__trainTestMap['testingSet_inFeatures_reShaped'])
                
        scoringMetrics = Scoring_Functions(testDataPrediction, self.__trainTestMap['testingSet_outFeature'])
        results = scoringMetrics.scoring_results()
        results += '\n\nExecution time: '
        results += str(end - start)
        results += 'secs'
        self.__writeResultsToFile(resultsFile, results)
        print 'Results:\n' + results

        fig = plt.figure()
        fig.set_figwidth(figWidth)
        fig.set_figheight(figHeight)
        
        ax1 = fig.add_subplot(3, 1, 1)
        ax1.grid()
        ax1.set_title('Expected')
        ax1.plot(self.__trainTestMap['testingSet_outFeature'])

        ax2 = fig.add_subplot(3, 1, 2)
        ax2.grid()
        ax2.set_title('Prediction - With rounding')
        ax2.plot(testDataPrediction)

        ax3 = fig.add_subplot(3, 1, 3)
        ax3.grid()
        ax3.set_title('Prediction - Without rounding')
        ax3.plot(testDataPredictionRaw)

        if predictionFilePrefix is not None:
            pathToFile = self.__resultsRootDir + "/" + self.__testID + "/" + predictionFilePrefix
            plt.savefig(pathToFile)

        # Reset __model
        self.__model = None
        self.__trainTestMap = None
    
    ###########################################################################
    # Private methods
    def __createTestResultsDir(self):
        dirName = "%s/%s" % (self.__resultsRootDir, self.__testID)
        if os.path.exists(dirName):
            shutil.rmtree(dirName)
        os.makedirs(dirName)
        
    def __loadData(self):
        self.__eegHeader = np.genfromtxt(self.__trainDataFileName, delimiter=',', max_rows=1, dtype=str)
        self.__eegData = np.genfromtxt(self.__trainDataFileName, delimiter=',', skip_header=1)
        testDataSet = np.genfromtxt(self.__testDataFileName, delimiter=',', skip_header=1)
        
        self.__testDataIndex = self.__eegData.shape[0]
        self.__eegData = np.append(self.__eegData, testDataSet, axis=0)

    def __timeSeriesToSupervised(self, lagCount, useLaggedOutput):
        featureCount = len(self.__eegHeader) - 1

        cols, names = list(), list()
        dfInput = pd.DataFrame(self.__eegData[:, :-1])
        dfOutput = pd.DataFrame(self.__eegData[:, -1])
        
        for i in range(lagCount, 0, -1):
            cols.append(dfInput.shift(i))
            if useLaggedOutput:
                cols.append(dfOutput.shift(i))
            else:
                # Set the lagged output variable to a constant 0.5
                tmp = len(dfOutput.shift(i))
                cols.append(pd.DataFrame(np.ones(tmp) * 0.5))
            
            names += [('%s(t-%d)' % (self.__eegHeader[j], i)) for j in range(featureCount)]
            names += [('%s(t-%d)' % (self.__eegHeader[-1], i))]

        cols.append(dfInput)
        cols.append(dfOutput)
        names += [('%s(t)' % self.__eegHeader[j]) for j in range(featureCount)]
        names += [('%s(t)' % self.__eegHeader[-1])]

        agg = pd.concat(cols, axis=1)
        agg.columns = names

        # Drop rows containing NaN values
        agg.dropna(inplace=True)
        return agg
    
    def __addToHistory(self, newCmd):
        tmpStr = ''
        if self.__cmdCount > 0:
            tmpStr += '\n'
            for itr in range(self.__cmdCount):
                tmpStr += '+'
                
        tmpStr += newCmd
        self.__cmdHistory += tmpStr
        self.__cmdCount += 1
        
    def __writeResultsToFile(self, resFilename, results):
        pathToResultsFile = self.__resultsRootDir + "/" + self.__testID + "/" + resFilename
        resString = ''
        
        if os.path.exists(pathToResultsFile):
            resString = '\n\n-----------------------------------------------------------------\n'
            fileMode = 'a' # Append
        else:
            fileMode = 'w' # Create
        
        text_file = open(pathToResultsFile, fileMode)
        
        resString += 'Results:\n'
        resString += results
        resString += '\n\nCommand history:\n----------------\n'
        resString += self.__cmdHistory        

        text_file.write(resString)
        text_file.close()
    ###########################################################################

## Experimentation

In [84]:
testID = 1
print "TestID: ", testID

resultsFilename = 'TestResults.txt'

predictor = LSTMPredictor(testID
                          , trainFilename='EEGEyeState_Training.arff.csv'
                          , testFilename='EEGEyeState_Testing.arff.csv')


res = predictor.RemoveOutliers(upperLimit=5000)
predictor.NormalizeData()

lagValue = 1
predictor.CompileModel(lagCount=lagValue
               , neuronCount=50
               , dropOut=0.0
               , useLaggedOutput=True
               , modelArchitectureFilename='01_modelArchitecture'
               , lstmStackCount=5)

predictor.FitModel(epochCount=100
           , batchSize=500
           , resultsFile=resultsFilename
           , lossFuncFilePrefix='01_lossFile'
           , predictionFilePrefix='01_predictionFile'
           , verbosity=2)


TestID:  1
(4480, 15)
(10000, 14)
Epoch 1/1


ValueError: Cannot feed value of shape (1, 1, 14) for Tensor u'lstm_101_input:0', which has shape '(10000, 1, 14)'